In [2]:
import pandas as pd
from IPython.display import display

# CSV 파일 불러오기
value_file_path = r"C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\kor_value.csv"
ticker_file_path = r"C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\kor_ticker.csv"

# 각각의 파일 불러오기
data = pd.read_csv(value_file_path, encoding='cp949')
ticker_data = pd.read_csv(ticker_file_path, encoding='cp949')

# 종목코드를 문자열로 변환하기 전에 NaN 값을 빈 문자열로 대체
data['종목코드'] = data['종목코드'].fillna('').astype(str).str.zfill(6)
ticker_data['종목코드'] = ticker_data['종목코드'].fillna('').astype(str).str.zfill(6)

# 지표별 조건 필터링
psr_filtered = data[(data['지표'] == 'PBR') & (data['값'] <= 1) & (data['값'] > 0)]
per_filtered = data[(data['지표'] == 'PER') & (data['값'] <= 15) & (data['값'] > 0)]
pcr_filtered = data[(data['지표'] == 'DY') & (data['값'] >= 0.03)]

# 조건을 모두 만족하는 종목 코드 찾기
growth_stock_codes = set(psr_filtered['종목코드']) & set(per_filtered['종목코드']) & set(pcr_filtered['종목코드'])

# 조건을 만족하는 종목만 선택하고, 필요한 지표만 포함
filtered_data = data[(data['종목코드'].isin(growth_stock_codes)) & (data['지표'].isin(['PBR', 'PER', 'DY']))]

# 테이블 재구성: 종목코드, 기준일을 인덱스로 두고 지표를 열로 변환
pivoted_data = filtered_data.pivot_table(index=['종목코드', '기준일'], columns='지표', values='값').reset_index()

# 종목코드 기준으로 종목명을 병합
pivoted_data = pivoted_data.merge(ticker_data[['종목코드', '종목명']], on='종목코드', how='left')

# ROE 열 추가 (PBR / PER)
pivoted_data['ROE'] = pivoted_data['PBR'] / pivoted_data['PER']

# ROE 값이 10 이상인 종목만 필터링
pivoted_data = pivoted_data[pivoted_data['ROE'] >= 0.1]

# 원하는 열 순서로 재정렬
pivoted_data = pivoted_data[['종목코드', '종목명', '기준일', 'PBR', 'PER', 'DY', 'ROE']]

# 종목코드를 숫자로 변환한 후 오름차순 정렬
pivoted_data['종목코드'] = pivoted_data['종목코드'].astype(int)
pivoted_data = pivoted_data.sort_values(by='종목코드').reset_index(drop=True)

# 결과 테이블로 출력
display(pivoted_data)

# CSV 파일로 저장
output_file_path = r"C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\result.csv"
pivoted_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

,종목코드,종목명,기준일,PBR,PER,DY,ROE
0,1230,동국홀딩스,2024-07-03,0.1662,0.4557,0.0719,0.364714
1,1420,태원물산,2024-07-03,0.8409,1.8568,0.0343,0.452876
2,1450,현대해상,2024-07-03,0.4827,3.9773,0.0604,0.121364
3,1560,제일연마,2024-07-03,0.8896,8.0310,0.0346,0.110771
4,2020,코오롱,2024-07-03,0.1445,0.9651,0.0347,0.149725
...,...,...,...,...,...,...,...
64,284740,쿠쿠홈시스,2024-07-03,0.5842,4.1190,0.0338,0.141831
65,298540,더네이쳐홀딩스,2024-07-03,0.4721,4.3194,0.0338,0.109298
66,300720,한일시멘트,2024-07-03,0.5449,4.2250,0.0591,0.128970
67,306200,세아제강,2024-07-03,0.3600,2.7243,0.0632,0.132144


In [3]:
import pandas as pd

# CSV 파일 읽기
csv_file_path = r"C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\result.csv"
data = pd.read_csv(csv_file_path, delimiter="\t")  # 탭 구분자로 읽어오기

# JSON 파일로 저장하기
json_file_path = r"C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\result.json"
data.to_json(json_file_path, orient="records", force_ascii=False, indent=4)

print(f"JSON 파일이 '{json_file_path}' 이름으로 저장되었습니다.")


JSON 파일이 'C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\result.json' 이름으로 저장되었습니다.


In [4]:
import json
import pandas as pd

# JSON 파일 경로
input_file = r"C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\result.json"
output_file = r"C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\result.json"

# JSON 파일 읽기
with open(input_file, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# 문자열로 결합된 데이터를 분리하여 각 열로 변환
processed_data = []
for item in raw_data:
    values = list(item.values())[0].split(',')
    processed_data.append({
        "종목코드": int(values[0]),
        "종목명": values[1],
        "기준일": values[2],
        "PBR": float(values[3]),
        "PER": float(values[4]),
        "DY": float(values[5]),
        "ROE": float(values[6])
    })

# DataFrame으로 변환 후 JSON으로 저장
df = pd.DataFrame(processed_data)
df.to_json(output_file, orient="records", force_ascii=False, indent=4)

print(f"필드별로 분리된 JSON 파일이 '{output_file}'에 저장되었습니다.")

필드별로 분리된 JSON 파일이 'C:\Users\USER\Desktop\3학년 2학기\데이터처리프로그래밍\result.json'에 저장되었습니다.
